In [1]:
%load_ext autoreload
%autoreload 2
from Libs.load_data import ClassificationDataLoader,DataLoader, get_dataset_split
from Libs import flares_plot as fplt
from Libs.threshold import get_labels_physic, get_labels_KDE, get_labels_quantile, get_labels_quantile_on_run
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import backend as K

2023-03-08 22:57:14.722827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 22:57:14.864724: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 22:57:14.864748: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 22:57:15.782246: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data_loader = ClassificationDataLoader(run=1000, N=1000, s=0.5, t=0.01, d=0.2, m=1)
params = data_loader.get_params()
params

{'run': 1000,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [3]:
Xs, best_labels = data_loader.load_data()

Loading Data


100%|██████████| 1000/1000 [00:03<00:00, 288.17it/s]


Loading Labels
Labels Loaded


In [4]:
bindexes = data_loader.get_standard_indexes()
df_train, df_val, df_test = get_dataset_split(Xs, best_labels, bindexes, window_size=100, overlap_size=0, label_treshold=1,
                                      split_on_run=True, shuffle_run=False, shuffle_window=False,
                                      test_size = 0.2, val_size=0.2, get_validation=True, random_state=42)
df_train

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,1.097460,0.800273,0.762377,1.139235,1.553440,1.806599,1.645525,1.419684,...,0.211269,0.152283,0.167314,0.193723,0.220627,0.203987,0.250678,0.269018,0.249159,0
1,0.228656,0.251608,0.162705,0.133498,0.108928,0.142085,0.184496,0.200684,0.228820,0.240179,...,0.158942,0.114497,0.120570,0.146734,0.160511,0.174416,0.162416,0.183431,0.169348,0
2,0.139880,0.147707,0.131194,0.108431,0.075699,0.089714,0.082418,0.098292,0.149462,0.124675,...,0.390679,0.367069,0.286149,0.250748,0.233916,0.314238,0.273822,0.376699,0.343073,0
3,0.311468,0.377507,0.503605,0.546112,0.550019,0.559552,0.384622,0.320995,0.261761,0.258866,...,0.046544,0.045418,0.042112,0.042715,0.045806,0.066110,0.051933,0.045467,0.051471,0
4,0.048260,0.047994,0.053862,0.044990,0.048068,0.041157,0.034158,0.049034,0.048493,0.035842,...,1.377049,1.204380,1.285629,1.255421,0.832831,0.849710,0.877229,0.925528,0.756486,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0.894873,1.030731,0.898352,0.887923,0.801791,0.728138,0.638125,0.818890,1.011304,1.243419,...,0.294739,0.307752,0.397041,0.539180,0.515245,0.295248,0.387283,0.481980,0.494195,1
5396,0.593780,0.591490,0.582267,0.561396,1.192204,0.916150,1.287857,1.609656,2.182222,1.856388,...,0.979838,1.020965,0.931575,0.786338,0.811534,0.557148,0.476873,0.357465,0.342375,0
5397,0.298239,0.199687,0.171544,0.196387,0.210877,0.224681,0.274870,0.166389,0.159029,0.221091,...,0.230019,0.292738,0.276140,0.256893,0.202953,0.146163,0.159135,0.190019,0.189322,0
5398,0.188162,0.143434,0.168813,0.203865,0.289923,0.263040,0.213453,0.216481,0.198572,0.228049,...,0.130151,0.158475,0.181278,0.136805,0.130137,0.154444,0.136134,0.117829,0.121386,0


In [5]:
df_val

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,0.794841,0.961714,0.773422,0.858592,0.608158,0.743288,0.776504,0.832919,...,0.155015,0.167666,0.131993,0.141247,0.171151,0.165978,0.160206,0.196225,0.210396,1
1,0.247564,0.160814,0.228383,0.204594,0.167953,0.160725,0.111309,0.131544,0.155309,0.113408,...,1.834831,1.515792,1.320335,1.590202,2.182392,2.024116,2.212742,3.477812,3.714054,1
2,3.838522,4.537823,4.923792,3.788025,3.624347,3.192889,2.461976,2.277469,1.532509,1.551760,...,0.233499,0.261160,0.288443,0.339135,0.306360,0.334486,0.255487,0.211829,0.159916,0
3,0.148766,0.141664,0.105070,0.126404,0.107553,0.110005,0.090352,0.075536,0.090373,0.122284,...,0.521858,0.460438,0.529690,0.547569,0.570987,0.655388,0.586328,0.667996,0.564026,1
4,0.579348,0.698647,0.712695,0.784738,0.723491,0.611856,0.656450,0.591715,0.446420,0.562588,...,1.201690,0.862037,0.891325,0.829132,1.359704,0.914465,0.816779,0.961132,1.099260,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.125542,0.137061,0.156362,0.128312,0.083011,0.077568,0.084634,0.130345,0.127974,0.118603,...,0.056108,0.047488,0.052894,0.054968,0.061461,0.057568,0.051230,0.050639,0.046315,0
1796,0.053604,0.070879,0.059345,0.075383,0.097257,0.093403,0.121281,0.151271,0.122940,0.103515,...,0.028792,0.022094,0.017086,0.015039,0.018051,0.015451,0.022964,0.031795,0.027705,0
1797,0.027338,0.021961,0.025480,0.023343,0.022380,0.019284,0.020583,0.032392,0.023834,0.018174,...,0.020185,0.020176,0.025927,0.026278,0.032224,0.032248,0.038812,0.038699,0.030643,0
1798,0.037464,0.039266,0.037234,0.027514,0.022862,0.027127,0.019360,0.016040,0.018135,0.015881,...,0.064680,0.105029,0.142267,0.147179,0.162281,0.122990,0.151431,0.193710,0.161193,0


In [6]:
df_test

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,future_flare
0,1.000000,0.975000,0.864849,0.940855,0.797112,1.136182,1.312568,1.293183,1.708106,1.992753,...,0.043055,0.038731,0.038268,0.028723,0.029924,0.031991,0.036987,0.041331,0.053353,0
1,0.051904,0.041102,0.043459,0.053803,0.065880,0.052029,0.058534,0.053135,0.050545,0.057436,...,0.042048,0.053979,0.046946,0.054593,0.065051,0.061722,0.072057,0.082646,0.057414,0
2,0.057048,0.048548,0.065221,0.061342,0.056631,0.085187,0.076338,0.066166,0.072643,0.040972,...,0.062960,0.071893,0.069668,0.069253,0.063292,0.054340,0.054102,0.043178,0.027668,0
3,0.049637,0.047778,0.034671,0.028317,0.038247,0.039409,0.032843,0.034893,0.029783,0.033671,...,0.051026,0.044779,0.054288,0.064697,0.061627,0.075575,0.062786,0.067390,0.050168,0
4,0.042756,0.043695,0.038555,0.053060,0.063615,0.045082,0.048137,0.047195,0.042249,0.040673,...,0.670718,0.655615,0.710235,0.594021,0.593149,0.681582,1.285178,1.356869,0.961953,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,0.033149,0.032264,0.033706,0.041612,0.046956,0.047962,0.045340,0.042412,0.049200,0.054696,...,0.054186,0.065986,0.066926,0.088580,0.107406,0.148387,0.187585,0.190281,0.248975,0
1796,0.213471,0.265604,0.250802,0.302429,0.391600,0.316887,0.437321,0.467344,0.453241,0.407954,...,0.081030,0.095245,0.097329,0.147589,0.144182,0.189755,0.169573,0.161331,0.113477,0
1797,0.137755,0.166856,0.211139,0.140109,0.133311,0.111607,0.146526,0.155874,0.144841,0.104751,...,0.089023,0.097040,0.077349,0.056044,0.070104,0.059120,0.102902,0.113795,0.102911,0
1798,0.071844,0.074584,0.071912,0.074552,0.073228,0.072019,0.093640,0.061726,0.067374,0.077197,...,0.025430,0.024417,0.021180,0.031455,0.029070,0.025637,0.031822,0.033097,0.031961,0


In [7]:
train_counts = df_train['future_flare'].value_counts()
train_counts

0    4476
1     924
Name: future_flare, dtype: int64

In [8]:
val_counts = df_val['future_flare'].value_counts()
val_counts

0    1479
1     321
Name: future_flare, dtype: int64

In [9]:
test_counts = df_test['future_flare'].value_counts()
test_counts

0    1478
1     322
Name: future_flare, dtype: int64

In [10]:
total_counts = train_counts.add(val_counts).add(test_counts)
total_counts

0    7433
1    1567
Name: future_flare, dtype: int64

Following this tensorflow tutorial in order to better train the neural architectures for unbalanced datasets
https://www.tensorflow.org/tutorials/structured_data/imbalanced_data?hl=en

In [11]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print(X_train.shape, X_val.shape, X_test.shape, X.shape)
print(y_train.shape, y_val.shape, y_test.shape, y.shape)

(5400, 100) (1800, 100) (1800, 100) (9000, 100)
(5400,) (1800,) (1800,) (9000,)


In [12]:
# Standardize Data
scaler = StandardScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
# Make the data uniform to multivariate timeseries
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# get automatically the number of classes
num_classes = len(np.unique(y))

In [13]:
def make_model(input_shape, num_classes, output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
    
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    if num_classes==2:
        output_layer = keras.layers.Dense(1, activation="sigmoid", bias_initializer=output_bias)(gap)
    else:
        output_layer = keras.layers.Dense(num_classes, activation="softmax", bias_initializer=output_bias)(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

initial_bias = np.log([total_counts[1]/total_counts[0]])
model = make_model(input_shape=X_train.shape[1:], num_classes=num_classes, output_bias=initial_bias)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 100, 64)           256       
                                                                 
 batch_normalization (BatchN  (None, 100, 64)          256       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 100, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 64)           12352     
                                                                 
 batch_normalization_1 (Batc  (None, 100, 64)          256       
 hNormalization)                                             

2023-03-08 22:57:55.133005: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-08 22:57:55.133069: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-08 22:57:55.133094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5166cf34c918): /proc/driver/nvidia/version does not exist
2023-03-08 22:57:55.133419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


                                                                 
 batch_normalization_2 (Batc  (None, 100, 64)          256       
 hNormalization)                                                 
                                                                 
 re_lu_2 (ReLU)              (None, 100, 64)           0         
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 25,793
Trainable params: 25,409
Non-trainable params: 384
_________________________________________________________________
_________________________________________________________________


In [14]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [15]:
epochs = 500
batch_size = 256 # ~ 20 iteration per epoch
# batch_size = 128
# batch_size = 64 # ~ 90 iteration per epoch

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "CNN_best_weights.h5", save_weights_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
if num_classes==2:
    loss = keras.losses.BinaryCrossentropy()
else:
    loss = "sparse_categorical_crossentropy"
model.compile(
    optimizer="adam",
    loss=loss,
    metrics=[f1_m],
)
history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

In [16]:
metric = "loss"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.yscale('log')
plt.show()
plt.close()

In [22]:
model = make_model(input_shape=X_train.shape[1:], num_classes=num_classes, output_bias=initial_bias)
model.load_weights("CNN_best_weights.h5")
y_test_pred = model.predict(X_test)
y_test_pred[y_test_pred>0.5] = 1
y_test_pred[y_test_pred<0.5] = 0
test_f1_macro = f1_score(y_test, y_test_pred, average='macro')
# test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test F1 Macro", test_f1_macro)

57/57 [==============================] - 1s 6ms/step
Test F1 Macro 0.7295550236726708
